# Correlating Returns

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [2]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [3]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                    
                             open    high     low    close     volume
2021-01-25 00:00:00-05:00  143.07  145.09  136.54  142.900  142335837
2021-01-26 00:00:00-05:00  143.60  144.30  141.37  143.185   85816874
2021-01-27 00:00:00-05:00  143.43  144.30  140.41  142.530  110342303
2021-01-28 00:00:00-05:00  139.52  141.99  136.70  137.040  127192871
2021-01-29 00:00:00-05:00  135.83  136.74  130.21  131.830  162479747

In [4]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-01-25,142.900
2021-01-26,143.185
2021-01-27,142.530
2021-01-28,137.040
2021-01-29,131.830


In [5]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2021-01-26,0.001994
2021-01-27,-0.004575
2021-01-28,-0.038518
2021-01-29,-0.038018
2021-02-01,0.017219


In [6]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [ ]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

In [ ]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

In [ ]:
# Get third topic
economy_headlines, _ = get_headlines("economy")


In [10]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [11]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [12]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)


In [13]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
    }
)

In [14]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [15]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
display(topic_sentiments)

,close,aapl_avg,trade_avg,economy_avg
2020-12-03,-0.001137,-0.090090,0.155805,0.199570
2020-12-04,-0.005694,0.008565,-0.032140,0.159280
2020-12-07,0.012762,0.126770,0.132200,-0.083920
2020-12-08,0.004281,0.082060,0.187675,0.128210
2020-12-09,-0.021395,0.033005,-0.018410,0.031285
2020-12-10,0.012739,0.145985,0.107165,0.179785
2020-12-11,-0.005924,0.098565,0.047545,0.048705
2020-12-14,-0.005878,0.089605,-0.005755,-0.021345
2020-12-15,0.050259,0.240600,0.185420,0.185560
2020-12-16,-0.001095,0.072350,0.098430,0.086070


In [16]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg
close,1,0.547862,0.455142,-0.0524333
aapl_avg,0.547862,1,-0.0109107,-0.425666
trade_avg,0.455142,-0.0109107,1,0.170973
economy_avg,-0.0524333,-0.425666,0.170973,1
